<a href="https://colab.research.google.com/github/Nithin2700000/Project-2/blob/main/Copy_of_KD_doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gensim
import gensim.downloader as api
print(list(api.info()['models'].keys()))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
wv = api.load('word2vec-google-news-300')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Reshape

# Read the Excel file
dataset = pd.read_excel('/content/Final doctor (3).xlsx')

# Access the data
# For example, you can print the first few rows
print(dataset.head())

# Assuming you have the dataset in two separate lists: reviews and ratings
reviews = dataset['review']
knowledge = dataset['knowledge']
Helpful = dataset['Helpful']
staff = dataset['staff']

# Normalize the ratings to a range of 0 to 4 using LabelEncoder
label_encoder = LabelEncoder()
knowledge_encoded = label_encoder.fit_transform(knowledge)  # Encode knowledge ratings from 0 to 4
helpful_encoded = label_encoder.fit_transform(Helpful)  # Encode Helpful ratings from 0 to 4
staff_encoded = label_encoder.fit_transform(staff)  # Encode staff ratings from 0 to 4

# Combine the encoded arrays into a single target array
target = np.stack((knowledge_encoded, helpful_encoded, staff_encoded), axis=1)

# Perform text preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews)  # Build vocabulary from the reviews

# Convert text reviews to sequences of indices
sequences = tokenizer.texts_to_sequences(reviews)

# Pad sequences to have the same length
max_seq_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_seq_length)
# Load the Google Word2Vec model
#path_to_model = r"C:\Users\adith\Downloads\word2vec_model.bin"
#word2vec_model = KeyedVectors.load_word2vec_format(output, binary=True)

# Create an embedding matrix
embedding_dim = 300  # Dimension of the Google Word2Vec model
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, index in tokenizer.word_index.items():
    if word in wv:
        embedding_matrix[index] = wv[word]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, target, test_size=0.2, random_state=42)


                                              review  knowledge  Helpful  \
0  b'Fabulous doctor! Dr. Ranen really takes his ...          5        5   
1  b'Wonderful doctor! Very punctual, but takes h...          5        5   
2  b"Great caring Doctor always there for you.Won...          5        5   
3  b'Dr. Foreman is great!!!! I believe he would ...          5        5   
4  b'Dr. Foreman I highly respect. He has done ma...          5        5   

   staff  
0      5  
1      5  
2      5  
3      5  
4      5  


In [ ]:
X_train.shape

(37322, 1678)

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model=load_model('/content/drive/MyDrive/model_doctor.h5')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1678, 300)         14235600  
                                                                 
 conv1d (Conv1D)             (None, 1674, 128)         192128    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 837, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 834, 128)          65664     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 417, 128)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 413, 128)          8

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


292/292 [==============================] - 11s 13ms/step - loss: 0.7917 - accuracy: 0.8848
Test Loss: 0.7917
Test Accuracy: 0.8848


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
tf.random.set_seed(3)

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))


        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results
    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        #student_predictions = tf.reshape(student_predictions, [-1, 3])


        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Flatten, Dense, Reshape

# Define the parameters


# Create the student model
model1= Sequential()
model1.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))

# Embedding layer

# Convolutional layers
filters = 64
model1.add(Conv1D(filters, 5, activation='relu'))
model1.add(MaxPooling1D())
model1.add(Conv1D(filters, 4, activation='relu'))
model1.add(MaxPooling1D())
model1.add(Conv1D(filters, 5, activation='relu'))
model1.add(MaxPooling1D())
model1.add(Dropout(0.2))
model1.add(Flatten())

# Dense layers
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(64, activation='relu'))

# Output layer
output_units = 3
num_classes = 5
model1.add(Dense(output_units * num_classes, activation='softmax'))
model1.add(Reshape((output_units, num_classes)))

# Compile the model
#model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1678, 300)         14235600  
                                                                 
 conv1d (Conv1D)             (None, 1674, 64)          96064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 837, 64)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 834, 64)           16448     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 417, 64)          0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 413, 64)           2

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, Dense, Reshape, GlobalMaxPooling1D

In [ ]:
student_model = Sequential()
student_model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False))
student_model.add(Conv1D(128, 3, activation='relu'))
student_model.add(GlobalMaxPooling1D())
student_model.add(Dense(256, activation='relu'))
student_model.add(Dense(output_units * num_classes, activation='softmax'))
student_model.add(Reshape((output_units, num_classes)))

In [ ]:
student_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1678, 300)         14235600  
                                                                 
 conv1d_3 (Conv1D)           (None, 1676, 128)         115328    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_3 (Dense)             (None, 256)               33024     
                                                                 
 dense_4 (Dense)             (None, 15)                3855      
                                                                 
 reshape_1 (Reshape)         (None, 3, 5)              0         
                                                      

In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=40,
)

# Distill teacher to student
distiller.fit(X_train, y_train, epochs=75)
print("---Testing Accuracy---")
# Evaluate student on test dataset
distiller.evaluate(X_test, y_test)

Epoch 1/75
1167/1167 [==============================] - 35s 26ms/step - sparse_categorical_accuracy: 0.8704 - student_loss: 1.3956 - distillation_loss: 3.7914e-06
Epoch 2/75
1167/1167 [==============================] - 29s 25ms/step - sparse_categorical_accuracy: 0.8718 - student_loss: 1.3932 - distillation_loss: 3.2681e-06
Epoch 3/75
1167/1167 [==============================] - 29s 25ms/step - sparse_categorical_accuracy: 0.8718 - student_loss: 1.3921 - distillation_loss: 3.5961e-06
Epoch 4/75
1167/1167 [==============================] - 29s 25ms/step - sparse_categorical_accuracy: 0.8718 - student_loss: 1.3905 - distillation_loss: 4.0055e-06
Epoch 5/75
1167/1167 [==============================] - 29s 25ms/step - sparse_categorical_accuracy: 0.8718 - student_loss: 1.3889 - distillation_loss: 4.0827e-06
Epoch 6/75
1167/1167 [==============================] - 29s 25ms/step - sparse_categorical_accuracy: 0.8718 - student_loss: 1.3875 - distillation_loss: 3.9922e-06
Epoch 7/75
1167/1167 [

[0.9074768424034119, 1.3708158731460571]

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb

In [ ]:
wandb.init()

wandb: Currently logged in as: nithin-270103 (nithinsairam). Use `wandb login --relogin` to force relogin
